<img width="50" src="https://carbonplan-assets.s3.amazonaws.com/monogram/dark-small.png" style="margin-left:0px;margin-top:20px"/>

# Buffer Pool Analysis

_by Grayson Badgley, September 17, 2020_

This notebook performs analysis of the
[California Air Resources Board carbon credit issuance database](https://ww2.arb.ca.gov/our-work/programs/compliance-offset-program/arb-offset-credit-issuance)
to quantify the number of ARB offset credits issued to the Warm Springs Phase I
(ACR260/CAFR5214) improved forest management offset project.

## Methodology

The Analysis consists of two parts: cleaning the data and calculating simple
summary statistics from the CARB issuance table.

## References

- CARB issuance database. Available online
  [https://ww3.arb.ca.gov/cc/capandtrade/offsets/issuance/arboc_issuance.xlsx].
- CARB quarterly compliance report (Q2 2020). Available online
  [https://ww2.arb.ca.gov/sites/default/files/2020-07/2020_q2_complianceinstrumentreport.pdf].


In [ ]:
import intake
import numpy as np
import pandas as pd
from carbonplan_styles.colors import colors
from carbonplan_styles.mpl import set_theme

# set options
set_theme(style="carbonplan_light")
c = colors("carbonplan_light")

### Load data


In [ ]:
df = pd.read_excel(
    "https://carbonplan-articles.s3.us-west-2.amazonaws.com/offset-project-fire/arboc_issuance.xlsx",
    sheet_name=3,
    engine="openpyxl",
)

display(df.head())

# Cleaning the data

We note that two of the the Forest projects are reforestation and have "Forest
Buffer Contributions" of "reforest defer" -- they haven't been issued any
credits so we will exclude them from the rest of the analysis.

We also note that one project has received zero credits -- so we will also
remove that one in order to be conservative.

Finally, we subset the table to just forests, because we want to understand
those projects and their relationship to the _forest_ buffer pool.


In [ ]:
df[df["Forest Buffer Account Contribution"].apply(lambda x: isinstance(x, str))]

In [ ]:
df = df[~df["Forest Buffer Account Contribution"].apply(lambda x: isinstance(x, str))]
df = df[df["ARB Offset Credits Issued"] > 0]
forest_df = df[df["Project Type"] == "Forest"]

display(forest_df)

## Early Action/Compliance

We note that the Early Action (EA) and Compliance (COP) contriubte to buffer
pool, so our default assumption is to include both project types in the
analysis. However, we also know that some projects converted over from the Early
Action program to the Compliance phase. Because we want to make sure we don't
double count, we'll check to see if any OPR Project IDs are listed as _both_ EA
and COP:


In [ ]:
display(forest_df["Early Action/ Compliance"].unique())

In [ ]:
display(
    forest_df.groupby("OPR Project ID")["Early Action/ Compliance"]
    .nunique()
    .value_counts()
)

## Avoid Double Counting

So, as we can see above, all projects just have one categorization: either EA or
COP. To ensure we don't double count a project, we'll remove EA projects from
the analysis.


In [ ]:
forest_df = forest_df[forest_df["Early Action/ Compliance"] == "COP"]
display(forest_df)

## Total number of forest projects

Having removed a) early action projects and b) projects that have not been
issued ARB Offset Credits, there are 98 forest carbon projects in the issuance
database


In [ ]:
n_projects = forest_df["OPR Project ID"].nunique()
print("Number of CARB Forest Projects: %d" % n_projects)

# Warm Springs Phase I (ACR260/CAFR5214)

We're interested in understanding how many credits have been issued to ACR260,
the project in Oregon that the Lionshead Fire burned. Let's take a closer look
at the records asscoiated with `OPR PROJECT ID == 'ACR260'`.

The ARB excel file records three reporting periods during which ACR260 was
issued credits. For each entry below, I double checked the
`Reporting Period Start Date`, `Reporting Period End Date`, and
`ARB Offset Credits Issued` against the publically available project
documentation from the
[ACR web portal](https://acr2.apx.com/mymodule/reg/TabDocuments.asp?r=111&ad=Prpt&act=update&type=PRO&aProj=pub&tablename=doc&id1=260).
The values listed below exactly match the project ACR Annual OPDR documentation.
It's worth noting that ACR260 also reported activity for a reporting period
starting 2016-09-28 and ending 2017-09-27, but no credits were ultimately issued
(which is noted in the issuance database, but not shown here because we excluded
all entries with `ARB Offset Credits Issued <= 0`).


In [ ]:
proj_id = "ACR260"
forest_df[forest_df["OPR Project ID"] == proj_id]

## How big is ACR260?

ACR260 is a pretty big project -- it's the 14th largest of the 98 compliance
period forest projects contained with the issuance database. ACR260 is also the
largest in forest project in Oregon.


In [ ]:
total_credits = forest_df["ARB Offset Credits Issued"].sum()
per_proj_credits = forest_df.groupby("OPR Project ID")[
    "ARB Offset Credits Issued"
].sum()

perc_credits = per_proj_credits / total_credits * 100

proj_state_map = (
    forest_df.groupby("OPR Project ID").State.max().to_dict()
)  # max() makes 1:1 mapping of opr_id to state
perc_credits = perc_credits.sort_values(ascending=False).to_frame()

perc_credits["state"] = perc_credits.index.map(proj_state_map)
perc_credits = perc_credits.rename(
    # clean up column title
    columns={"ARB Offset Credits Issued": "Percent Forest ARB Offset Credits Issued"}
)

proj_loc = perc_credits.index.get_loc(proj_id)
display(perc_credits[: proj_loc + 1])  # +1 to include ACR260, otherwise off by one

While deriving the above ordering, I realized that it's absolutely the case that
a big project could have occured during the Early Action period and potentially
affect the ordering shown above. To check this, let's just quick re-do the above
table but this time with both EA and COP projects. As it turns out, CAR730 was a
big project in California that doesn't seem to have transfered over to the
complicance period. So we can still say that ACR260 is the biggest in Oregon,
but it's probably better to say that its "among the 15 largest" forest projects!


In [ ]:
per_proj_credits = (
    df.loc[df["Project Type"] == "Forest"]
    .groupby("OPR Project ID")["ARB Offset Credits Issued"]
    .sum()
)

proj_state_map = (
    df.groupby("OPR Project ID").State.max().to_dict()
)  # max() makes 1:1 mapping of opr_id to state
sorted_credits = per_proj_credits.sort_values(ascending=False).to_frame()

sorted_credits["state"] = sorted_credits.index.map(proj_state_map)
sorted_credits = sorted_credits.rename(
    # clean up column title
    columns={"ARB Offset Credits Issued": "Percent Forest ARB Offset Credits Issued"}
)

proj_loc = sorted_credits.index.get_loc(proj_id)
display(sorted_credits[: proj_loc + 1])  # +1 to include ACR260, otherwise off by one

## Size relative to the buffer pool

The next exercise is to understand the credits issued to ACR260 in context of
the _remaining_ credits within the buffer pool. The issuance database lists all
ARB offset credits issued and the number of those credits that are set aside in
the forest buffer account. So summing `Forest Buffer Account Contribution` would
give us the _total_ number of credits ever deposited in the buffer pool.
However, the buffer pool has been drawn upon in the past. So for our statistics,
we're going to use the Q2 complicance report mentioned above. Footnote `+`
reports that the forest buffer account, as of Q2 2020, had 24,079,774 credits.

From there, it's just a matter of summing up `ARB Offset Credits Issued` for
ACR260


In [ ]:
total_buffer_pool = 24_079_774
proj_credits = forest_df[forest_df["OPR Project ID"] == proj_id][
    "ARB Offset Credits Issued"
].sum()
proj_frac = proj_credits / total_buffer_pool * 100

print(f"Remaining buffer pool (Q2 2020): {total_buffer_pool}")
print(f"{proj_id} Total Issued Credits: {proj_credits}")
print(f"ACR260 as a fraction of Buffer Pool: {proj_frac:.2f}%")

## Double Check Offset Issued vs Buffer Pool

We also wanted to make sure that `ARB Offset Credits Issued` represented 100
percent of the credits generated by the project, including both the credits
awarded to the offset project operator and the credits reserved for the buffer
pool. To double check that this thinking is correct, we calculate
`buffer_contrib` as the ratio of `Forest Buffer Account Contribution` and
`ARB Offset Credits Issued`. If `ARB Offset Credits Issued` represents the full
allocation of credits generated by a project (prior to some credits being set
aside for the buffer pool), that ratio should range mostly between 12 and 20
percent, which is indeed the case.


In [ ]:
forest_df.loc[:, "buffer_contrib"] = (
    forest_df["Forest Buffer Account Contribution"]
    / forest_df["ARB Offset Credits Issued"]
).copy()
display(forest_df["buffer_contrib"].astype(float).describe())

In [ ]:
g = forest_df["buffer_contrib"].hist(color=c["red"])
g.grid(None)
g.spines["top"].set_visible(False)
g.spines["right"].set_visible(False)